# Steganografia - testy narzędzi
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb](https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł
- Anton Maisiuk

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - Ocena narzędzi steganograficznych 

Na podstawie listy ze strony [Wikipedii](https://en.wikipedia.org/wiki/Steganography_tools) wybierz 5 narzędzi steganograficznych i dokonaj ich oceny (badań):
- porównaj szybkość działania dla tego samego pliku,
- jeżeli to możliwe, to oceń ilość/jakość zmian pliku źródłowego.


## Zadanie 2 - Własny system steganograficzny

Wykonaj prosty system steganograficzny plików graficznych BMP polegający na osadzaniu w najmniej znaczących bitach danego stegotekstu:
- ilość bitów ma być zmienna, od 1 do 4,
- dokonaj oceny wizualnej głębokości zmian pliku źródłowego w zależności od ilości bitów modyfikowanych.

In [54]:
function* getBitsIterator(bytes: Uint8Array, bits: 1 | 2 | 3 | 4) {
    switch (bits) {
        case 1:
            for (const byte of bytes) {
                yield (byte & 0b10000000) >> 7;
                yield (byte & 0b01000000) >> 6;
                yield (byte & 0b00100000) >> 5;
                yield (byte & 0b00010000) >> 4;
                yield (byte & 0b00001000) >> 3;
                yield (byte & 0b00000100) >> 2;
                yield (byte & 0b00000010) >> 1;
                yield (byte & 0b00000001);
            }
            break;
        case 2:
            for (const byte of bytes) {
                yield (byte & 0b11000000) >> 6;
                yield (byte & 0b00110000) >> 4;
                yield (byte & 0b00001100) >> 2;
                yield (byte & 0b00000011);
            }
            break;
        case 3:
            {
                // Holds the current bits buffer
                let buffer = 0; 
                 // Number of bits available in the buffer
                let bufferLength = 0;

                for (const byte of bytes) {
                    // Add new byte to the buffer
                    buffer = (buffer << 8) | byte;
                    bufferLength += 8;

                    // Emit chunks as long as we can get 3 bits
                    while (bufferLength >= 3) {
                        yield (buffer >> (bufferLength - 3)) & 0b111; 
                        bufferLength -= 3;
                    }
                }

                // Handle leftover bits (if any) by padding with zeros
                if (bufferLength > 0) {
                    yield (buffer << (3 - bufferLength)) & 0b111; // Pad with zeros
                }
            }
            break;
        case 4:
            for (const byte of bytes) {
                yield (byte & 0b11110000) >> 4;
                yield (byte & 0b00001111);
            }
            break;
    }
}

In [55]:
//@deno-types="npm:@types/bmp-js"
import { decode, encode } from "npm:bmp-js";
import type { Buffer } from "node:buffer";

function embedText(inputFile: Buffer, text: string, bits: 1 | 2 | 3 | 4) {
    const bmpData = decode(inputFile);
    const textEncoder = new TextEncoder();
    const textBytes = textEncoder.encode(text + "\0");
    const bitsIterator = getBitsIterator(
        textBytes,
        bits,
    );
    let i = 0;
    const mask = 255 << bits;
    for (const chunk of bitsIterator) {
        // don't write to alpha channel
        if (!(i % 4)) i++;

        bmpData.data[i] = (bmpData.data[i] & mask) | chunk;
        i++;
    }
    return encode(bmpData);
}

function readEmbeddedText(inputFile: Buffer, bits: 1 | 2 | 3 | 4) {
    const bmpData = decode(inputFile);
    const mask = 255 >> (8 - bits);

    const readBytes: number[] = [];
    let chunkBuffer = 0;
    let chunkBufferLength = 0;
    let i = -1;
    for (const byte of bmpData.data) {
        // don't read from alpha channel
        if (!(++i % 4)) continue;

        const chunk = byte & mask;

        chunkBuffer = chunkBuffer << bits;
        chunkBuffer = chunkBuffer | chunk;
        chunkBufferLength += bits;

        // if there is already full byte in buffer
        if (chunkBufferLength >= 8) {
            //get full byte
            const overflowBits = chunkBufferLength - 8;
            const fullByte = chunkBuffer >> overflowBits;

            //left in buffer only overflowed bits
            chunkBuffer &= 255 >> (8 - overflowBits);
            chunkBufferLength = overflowBits;

            //write fullbyte to result array
            readBytes.push(fullByte);

            // end if read character is '/0'
            if (fullByte === 0) {
                break;
            }
        }
    }

    const textDecoder = new TextDecoder();
    return textDecoder.decode(new Uint8Array(readBytes)).slice(0, -1);
}

In [ ]:
import fs from "node:fs";

const bmpBuffer = fs.readFileSync("cat.bmp");
const bmpModified = embedText(
    bmpBuffer,
    "To jest jakiś dłuższy tekst. Czy widać różnicę?",
    3,
);

fs.writeFileSync("catModified.bmp", bmpModified.data);
const readFile = fs.readFileSync("catModified.bmp");
readEmbeddedText(readFile, 3);


"To jest jakiś dłuższy tekst. Czy widać różnicę? No bo nie "

In [ ]:
const beeMovieScript = await fetch("https://gist.githubusercontent.com/MattIPv4/045239bc27b16b2bcf7a3a9a4648c08a/raw/2411e31293a35f3e565f61e7490a806d4720ea7e/bee%2520movie%2520script").then(r => r.text())
beeMovieScript

"According to all known laws of aviation, there is no way a bee should be able to fly.\n" +
  "Its wings are too small to get its fat little body off the ground.\n" +
  "The bee, of course, flies anyway because bees don't care what humans think is impossible.\n" +
  "Yellow, black. Yellow, black. Yellow, black. Yellow, black.\n" +
  "Ooh, black and yellow!\n" +
  "Let's shake it up a little.\n" +
  "Barry! Breakfast is ready!\n" +
  "Coming!\n" +
  "Hang on a second.\n" +
  "Hello?\n" +
  "Barry?\n" +
  "Adam?\n" +
  "Can you believe this is happening?\n" +
  "I can't.\n" +
  "I'll pick you up.\n" +
  "Looking sharp.\n" +
  "Use the stairs, Your father paid good money for those.\n" +
  "Sorry. I'm excited.\n" +
  "Here's the graduate.\n" +
  "We're very proud of you, son.\n" +
  "A perfect report card, all B's.\n" +
  "Very proud.\n" +
  "Ma! I got a thing going here.\n" +
  "You got lint on your fuzz.\n" +
  "Ow! That's me!\n" +
  "Wave to us! We'll be in row 118,000.\n" +
  "Bye!\n" 

In [ ]:
const bmpBuffer = fs.readFileSync("cat.bmp");
const bmpModified = embedText(
    bmpBuffer,beeMovieScript + beeMovieScript + beeMovieScript,
    4,
);

fs.writeFileSync("catModified.bmp", bmpModified.data);
const readFile = fs.readFileSync("catModified.bmp");
readEmbeddedText(readFile, 4);


"According to all known laws of aviation, there is no way a bee should be able to fly.\n" +
  "Its wings are too small to get its fat little body off the ground.\n" +
  "The bee, of course, flies anyway because bees don't care what humans think is impossible.\n" +
  "Yellow, black. Yellow, black. Yellow, black. Yellow, black.\n" +
  "Ooh, black and yellow!\n" +
  "Let's shake it up a little.\n" +
  "Barry! Breakfast is ready!\n" +
  "Coming!\n" +
  "Hang on a second.\n" +
  "Hello?\n" +
  "Barry?\n" +
  "Adam?\n" +
  "Can you believe this is happening?\n" +
  "I can't.\n" +
  "I'll pick you up.\n" +
  "Looking sharp.\n" +
  "Use the stairs, Your father paid good money for those.\n" +
  "Sorry. I'm excited.\n" +
  "Here's the graduate.\n" +
  "We're very proud of you, son.\n" +
  "A perfect report card, all B's.\n" +
  "Very proud.\n" +
  "Ma! I got a thing going here.\n" +
  "You got lint on your fuzz.\n" +
  "Ow! That's me!\n" +
  "Wave to us! We'll be in row 118,000.\n" +
  "Bye!\n" 